## Project Week07

Yilun Wu (yw528)

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import fsolve, minimize
import os
import sys
import inspect
import statsmodels.api as sm
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)
from lib.riskmgmt import return_calculate, compute_VaR, compute_ES
np.random.seed(545) # random seed for consistent result

### Problem 1

In [2]:
S = 151.03 # current stock price
X = 165 # strike price
curr_date = pd.to_datetime('2022-03-13') # current date
expiration_date = pd.to_datetime('2022-04-15') # expiration date
T = (expiration_date-curr_date).days/365 # time to maturity
r = 0.0425 # risk-free rate
q = 0.0053 # Continuous dividend rate
b = r-q # Cost of carry

sigma = 0.2 # predicted volatility value
option_types = ['Call', 'Put']
greek_names = ['delta', 'gamma', 'vega', 'theta', 'rho', 'carry_rho']

dividend = 0.88
dividend_date = pd.to_datetime('2022-04-11')
N = 200 # number of steps
dividend_time = int((dividend_date-curr_date).days/(expiration_date-curr_date).days*N)

#### Implement the Closed Form Greeks for GBSM

In [3]:
class GBSM:
    def __init__(self, S, X, T, sigma, r, b, option_type):
        self.S = S
        self.X = X
        self.T = T
        self.r = r
        self.b = b
        self.sigma = sigma
        self.option_type = option_type
        self.d1 = (np.log(S/X)+(b+sigma**2/2)*T)/(sigma*np.sqrt(T))
        self.d2 = self.d1-sigma*np.sqrt(T)

    def _delta(self):
        if self.option_type == 'Call':
            return np.exp((self.b-self.r)*self.T)*norm.cdf(self.d1)
        elif self.option_type == 'Put':
            return np.exp((self.b-self.r)*self.T)*(norm.cdf(self.d1)-1)
        else:
            raise ValueError("Option type must be either 'Call' or 'Put'")
    
    def _gamma(self):
        return norm.pdf(self.d1)*np.exp((self.b-self.r)*self.T)/(self.S*self.sigma*np.sqrt(self.T))

    def _vega(self):
        return self.S*np.exp((self.b-self.r)*self.T)*norm.pdf(self.d1)*np.sqrt(self.T)

    def _theta(self):
        if self.option_type == 'Call':
            return -self.S*np.exp((self.b-self.r)*self.T)*norm.pdf(self.d1)*self.sigma/(2*np.sqrt(self.T))-(self.b-self.r)*self.S*np.exp((self.b-self.r)*self.T)*norm.cdf(self.d1)-self.r*self.X*np.exp(-self.r*self.T)*norm.cdf(self.d2)
        elif self.option_type == 'Put':
            return -self.S*np.exp((self.b-self.r)*self.T)*norm.pdf(self.d1)*self.sigma/(2*np.sqrt(self.T))+(self.b-self.r)*self.S*np.exp((self.b-self.r)*self.T)*norm.cdf(-self.d1)+self.r*self.X*np.exp(-self.r*self.T)*norm.cdf(-self.d2)
        else:
            raise ValueError("Option type must be either 'Call' or 'Put'")

    def _rho(self): # Note: original formual assumes r=b, yet it does not hold in this case, so we redo the derivation
        if self.option_type == 'Call': # Call: rho = -T*S*exp((b-r)*T)*N(d1)+T*X*exp(-r*T)*N(d2)
            return -self.T*self.S*np.exp((self.b-self.r)*T)*norm.cdf(self.d1)+self.T*self.X*np.exp(-self.r*self.T)*norm.cdf(self.d2)
        elif self.option_type == 'Put': # Put: rho = T*S*exp((b-r)*T)*N(-d1)-T*X*exp(-r*T)*N(-d2)
            return self.T*self.S*np.exp((self.b-self.r)*T)*norm.cdf(-self.d1)-self.T*self.X*np.exp(-self.r*self.T)*norm.cdf(-self.d2)
        else:
            raise ValueError("Option type must be either 'Call' or 'Put'")
        
    def _carry_rho(self):
        if self.option_type == 'Call':
            return self.T*self.S*np.exp((self.b-self.r)*self.T)*norm.cdf(self.d1)
        elif self.option_type == 'Put':
            return -self.T*self.S*np.exp((self.b-self.r)*self.T)*norm.cdf(-self.d1)
        else:
            raise ValueError("Option type must be either 'Call' or 'Put'")

    def getAllGreeks(self):
        return {'delta': self._delta(), 'gamma': self._gamma(), 'vega': self._vega(), 'theta': self._theta(), 'rho': self._rho(), 'carry_rho': self._carry_rho()}

In [4]:
# compute values for both a call and a put
for option_type in option_types:
    gbsm = GBSM(S, X, T, sigma, r, b, option_type)
    allGreeks = gbsm.getAllGreeks()
    for greek_name in greek_names:
        print(f"{greek_name} of the {option_type} option is {allGreeks[greek_name]}")
    print()

delta of the Call option is 0.08297130333914773
gamma of the Call option is 0.016822916101852648
vega of the Call option is 6.938710929513443
theta of the Call option is -8.126522359668838
rho of the Call option is -0.030359909374904293
carry_rho of the Call option is 1.132953825011723

delta of the Put option is -0.9165496333661425
gamma of the Put option is 0.016822916101852648
vega of the Put option is 6.938710929513443
theta of the Put option is -1.9409914783019566
rho of the Put option is -1.2427313221864171
carry_rho of the Put option is -12.515271800549371



#### Implement a finite difference derivative calculation

In [5]:
# Functions for computing partial derivatives
def first_order_derivative(f, x, h): # f'(x) = (f(x+h)-f(x-h))/(2h)
    return (f(x+h)-f(x-h))/(2*h)

def second_order_derivative(f, x, h): # f''(x) = (f(x+h)+f(x-h)-2f(x))/h^2
    return (f(x+h)+f(x-h)-2*f(x))/(h**2)

def compute_partial_derivative(f, order, arg_name, h=1e-4):
    # initialize for argument names and order
    arg_names = list(inspect.signature(f).parameters.keys())
    derivative_fs = {1: first_order_derivative, 2: second_order_derivative}

    def partial_derivative(*args, **kwargs):
        # parse argument names and order
        args_dict = dict(list(zip(arg_names, args)) + list(kwargs.items()))
        arg_val = args_dict.pop(arg_name)

        def partial_f(x):
            p_kwargs = {arg_name:x, **args_dict}
            return f(**p_kwargs)
        return derivative_fs[order](partial_f, arg_val, h)
    return partial_derivative

# Function for computing Black-Scholes prices (values)
def black_scholes(S, X, T, sigma, r, b, option_type):
    d1 = (np.log(S/X)+(b+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    if option_type == 'Call':
        return S*np.exp((b-r)*T)*norm.cdf(d1)-X*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == 'Put':
        return X*np.exp(-r*T)*norm.cdf(-d2)-S*np.exp((b-r)*T)*norm.cdf(-d1)
    else:
        raise ValueError("Option type must be either 'Call' or 'Put'")

In [6]:
# compute values for both a call and a put
for option_type in option_types:
    greek_funcs = {
        'delta': compute_partial_derivative(black_scholes, 1, 'S'),
        'gamma': compute_partial_derivative(black_scholes, 2, 'S'),
        'vega': compute_partial_derivative(black_scholes, 1, 'sigma'),
        'theta': compute_partial_derivative(black_scholes, 1, 'T'),
        'rho': compute_partial_derivative(black_scholes, 1, 'r'),
        'carry_rho': compute_partial_derivative(black_scholes, 1, 'b')
    }
    for greek_name in greek_names:
        greek_value = greek_funcs[greek_name](S=S, X=X, T=T, sigma=sigma, r=r, b=b, option_type=option_type)
        if greek_name == 'theta': greek_value = -greek_value
        print(f"{greek_name} of the {option_type} option is {greek_value}")
    print()

delta of the Call option is 0.08297130338341674
gamma of the Call option is 0.016824763804379472
vega of the Call option is 6.938710350752331
theta of the Call option is -8.126520224163158
rho of the Call option is -0.030359909377608574
carry_rho of the Call option is 1.13295383685319

delta of the Put option is -0.9165496332741441
gamma of the Put option is 0.016834178495628294
vega of the Put option is 6.938710350681276
theta of the Put option is -1.9409893427280167
rho of the Put option is -1.242731322150803
carry_rho of the Put option is -12.515271789084181



#### Implement the binomial tree valuation for American options with and without discrete dividends

In [7]:
def binominal_tree_without_dividends(S, X, T, r, b, sigma, N, option_type):
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    pu = (np.exp(b*dt)-d)/(u-d)
    pd = 1.0-pu
    df = np.exp(-r*dt)
    z = 1 if option_type == 'Call' else -1
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
    nNodes = nNodeFunc(N)
    optionValues = np.empty(nNodes, dtype=float)

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i,j)
            price = S*(u**i)*(d**(j-i))
            optionValues[idx] = max(0, z*(price-X))
            if j < N:
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1, j+1)] + pd*optionValues[idxFunc(i, j+1)])  )
    return optionValues[0]

def binominal_tree_with_dividends(S, X, T, r, sigma, N, option_type, dividend_times=None, dividends=None):
    if not dividend_times or not dividends or dividend_times[0] > N:
        return binominal_tree_without_dividends(S, X, T, r, 0, sigma, N, option_type)

    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1.0/u
    pu = (np.exp(r*dt)-d)/(u-d)
    pd = 1.0-pu
    df = np.exp(-r*dt)
    z = 1 if option_type == 'Call' else -1
    
    def nNodeFunc(n):
        return (n+2)*(n+1)//2
    def idxFunc(i,j):
        return nNodeFunc(j-1)+i
   
    nNodes = nNodeFunc(dividend_times[0])

    optionValues = np.empty(nNodes, dtype=float)

    for j in range(dividend_times[0], -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = S*(u**i)*(d**(j-i))
            
            if j < dividend_times[0]:
                #times before the dividend working backward induction
                optionValues[idx] = max(0,z*(price-X))
                optionValues[idx] = max(optionValues[idx], df*(pu*optionValues[idxFunc(i+1,j+1)] + pd*optionValues[idxFunc(i,j+1)])  )
                
            else:
                val_no_ex = binominal_tree_with_dividends(price-dividends[0], X, T-dividend_times[0]*dt, r, sigma, N-dividend_times[0], option_type, [x-dividend_times[0] for x in dividend_times[1:]], dividend_times[1:])
                val_ex = max(0, z*(price-X))
                optionValues[idx] = max(val_no_ex, val_ex)

    return optionValues[0]

##### Without dividends

In [8]:
# Calculate the value of the call and the put
call_without_dividend = binominal_tree_without_dividends(S, X, T, r, b, sigma, N, option_type='Call')
print(f'Value of the Call without dividend: {call_without_dividend}')
put_without_dividend = binominal_tree_without_dividends(S, X, T, r, b, sigma, N, option_type='Put')
print(f'Value of the Put without dividend: {put_without_dividend}')

Value of the Call without dividend: 0.33600393488438995
Value of the Put without dividend: 14.036960189311927


In [9]:
# Calculate the Greeks of each (Which is equal to the closed form Greeks)
for option_type in option_types:
    gbsm = GBSM(S, X, T, sigma, r, b, option_type)
    allGreeks = gbsm.getAllGreeks()
    for greek_name in greek_names:
        print(f"{greek_name} of the {option_type} option is {allGreeks[greek_name]}")
    print()

delta of the Call option is 0.08297130333914773
gamma of the Call option is 0.016822916101852648
vega of the Call option is 6.938710929513443
theta of the Call option is -8.126522359668838
rho of the Call option is -0.030359909374904293
carry_rho of the Call option is 1.132953825011723

delta of the Put option is -0.9165496333661425
gamma of the Put option is 0.016822916101852648
vega of the Put option is 6.938710929513443
theta of the Put option is -1.9409914783019566
rho of the Put option is -1.2427313221864171
carry_rho of the Put option is -12.515271800549371



##### With dividends

In [10]:
# Calculate the value of the call and the put
call_with_dividend = binominal_tree_with_dividends(S, X, T, r, sigma, N, option_type='Call', dividend_times=[dividend_time], dividends=[dividend])
print(f'Value of the Call with dividend: {call_with_dividend}')
put_with_dividend = binominal_tree_with_dividends(S, X, T, r, sigma, N, option_type='Put', dividend_times=[dividend_time], dividends=[dividend])
print(f'Value of the Put with dividend: {put_with_dividend}')

Value of the Call with dividend: 0.2957566152561088
Value of the Put with dividend: 14.565662119188891


In [11]:
# Calculate the Greeks of each.
def compute_sensitivity_to_change(S, X, T, r, sigma, N, option_type, dividend_times, dividends, h):
    V1 = binominal_tree_with_dividends(S, X, T, r, sigma, N, option_type, dividend_times, [d+h for d in dividends])
    V2 = binominal_tree_with_dividends(S, X, T, r, sigma, N, option_type, dividend_times, [d-h for d in dividends])
    return (V1-V2)/(2*h)

for option_type in option_types:
    greek_funcs = {
        'delta': compute_partial_derivative(binominal_tree_with_dividends, 1, 'S'),
        'gamma': compute_partial_derivative(binominal_tree_with_dividends, 2, 'S'),
        'vega': compute_partial_derivative(binominal_tree_with_dividends, 1, 'sigma'),
        'theta': compute_partial_derivative(binominal_tree_with_dividends, 1, 'T'),
        'rho': compute_partial_derivative(binominal_tree_with_dividends, 1, 'r'),
    }
    
    for greek_name in greek_names[:-1]:
        greek_value = greek_funcs[greek_name](S=S, X=X, T=T, r=r, sigma=sigma, N=N, option_type=option_type, dividend_times=[dividend_time], dividends=[dividend])
        if greek_name == 'theta': greek_value = -greek_value
        print(f"{greek_name} of the {option_type} option is {greek_value}")
    sensitivity_to_change = compute_sensitivity_to_change(S, X, T, r, sigma, N, option_type, [dividend_time], [dividend], 1e-4)
    print(f"Sensitivity of the {option_type} option is {sensitivity_to_change}")
    print()

delta of the Call option is 0.06862091639747714
gamma of the Call option is -4.440892098500626e-08
vega of the Call option is 5.9130771620033435
theta of the Call option is -6.954221580864061
rho of the Call option is 0.8807123572601716
Sensitivity of the Call option is -0.024509121646643095

delta of the Put option is -0.9383203737467483
gamma of the Put option is 1.0658141036401503e-06
vega of the Put option is 5.491390789389428
theta of the Put option is -0.30317203930430026
rho of the Put option is -12.276805139892488
Sensitivity of the Put option is 0.9413040506256465



### Problem 2

In [12]:
# Read data from csv file
portfolio = pd.read_csv('problem2.csv')
portfolio['ExpirationDate'] = pd.to_datetime(portfolio['ExpirationDate'])
portfolio

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80
5,CallSpread,Option,AAPL,-1,Call,2023-04-21,160.0,2.21
6,PutSpread,Option,AAPL,1,Put,2023-04-21,150.0,4.85
7,PutSpread,Option,AAPL,-1,Put,2023-04-21,140.0,1.84
8,Stock,Stock,AAPL,1,NaN,NaT,NaN,151.03
9,Call,Option,AAPL,1,Call,2023-04-21,150.0,6.80


In [13]:
# Assumptions
S0 = 165 # Underlying price (Current AAPL stock price)
curr_date = pd.to_datetime('2023-03-03') # Current date
r = 0.0425 # Risk-free rate
dividend = 1.0
dividend_date = pd.to_datetime('2023-03-15')
N = 20 # number of steps

In [14]:
# Get names of all portfolios
portfolio_names = portfolio['Portfolio'].unique()
print("Number of portfolios: ", len(portfolio_names))
print("Portfolio names: ", portfolio_names)

Number of portfolios:  9
Portfolio names:  ['Straddle' 'SynLong' 'CallSpread' 'PutSpread' 'Stock' 'Call ' 'Put '
 'CoveredCall' 'ProtectedPut']


In [15]:
# Read data from csv file
DailyPrices = pd.read_csv('DailyPrices.csv')
DailyPrices

,Date,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,2/14/2022,432.011322,167.863144,292.261475,155.167007,291.920013,135.526001,135.300003,217.699997,242.443298,...,197.263107,64.592575,46.290192,151.118790,58.443172,377.068665,247.324020,48.182598,60.352272,66.789505
1,2/15/2022,438.978333,171.749573,297.680664,156.510498,307.476654,136.608505,136.425507,221.000000,264.702484,...,199.789520,64.328896,46.502743,159.029022,58.653572,372.440185,255.490829,49.446892,62.076630,67.673614
2,2/16/2022,439.470337,171.511032,297.333191,158.100494,307.796661,137.738007,137.487503,216.539993,264.862305,...,201.139511,64.172638,46.770744,165.211960,59.198696,377.000458,256.349976,50.203476,61.990410,65.915215
3,2/17/2022,430.082642,167.863144,288.626679,154.652496,292.116669,132.539002,132.308502,207.710007,244.841064,...,194.109894,64.514450,47.473091,157.448944,58.615318,378.822571,248.439911,48.600708,59.921181,64.019295
4,2/18/2022,427.297852,166.292648,285.846893,152.601502,285.660004,130.403000,130.467499,206.160004,236.199127,...,193.984528,64.455841,47.815025,157.380234,58.385788,376.571686,245.042847,48.381695,59.700851,64.981995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2/3/2023,412.350006,154.264465,258.350006,103.389999,189.979996,104.779999,105.220001,186.529999,211.000000,...,165.389999,65.910004,46.910000,178.820007,84.500000,459.079987,283.140015,41.130001,49.561237,80.222557
245,2/6/2023,409.829987,151.498688,256.769989,102.180000,194.759995,102.900002,103.470001,186.059998,210.889999,...,163.000000,66.169998,46.959999,177.550003,86.360001,469.100006,271.480011,41.340000,49.076408,79.853935
246,2/7/2023,415.190002,154.414230,267.559998,102.110001,196.809998,107.639999,108.040001,191.619995,221.729996,...,162.940002,65.080002,46.560001,181.020004,86.050003,468.329987,272.450012,41.400002,49.501869,79.565002
247,2/8/2023,410.649994,151.688400,266.730011,100.050003,201.289993,99.370003,100.000000,183.429993,222.050003,...,161.559998,64.790001,46.500000,177.759995,85.669998,469.649994,268.220001,41.570000,48.739998,80.309998


In [16]:
# Compute the log returns of AAPL and demean the series (so the mean is 0) 
AAPL_returns = return_calculate(DailyPrices, method="LOG", dateColumn="Date")['AAPL']
AAPL_returns -= AAPL_returns.mean()
assert(AAPL_returns.mean() < 1e-10) # mean should be 0
AAPL_returns

/Users/wuyilun/Desktop/FINTECH545/lib/riskmgmt.py:236: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:, i]


0      0.023325
1     -0.000953
2     -0.021062
3     -0.008963
4     -0.017536
         ...   
243    0.024543
244   -0.017655
245    0.019498
246   -0.017374
247   -0.006499
Name: AAPL, Length: 248, dtype: float64

#### Calculate Mean, VaR and ES using Simulated Prices from AAPL Returns

In [17]:
# Function for computing the implied volatility when using the binomial tree model
def compute_implied_volatility_binomial_tree(S, X, T, r, N, option_type, option_price, dividend_times=None, dividends=None):
    f1 = lambda sigma: (binominal_tree_with_dividends(S, X, T, r, sigma, N, option_type, dividend_times, dividends)-option_price)
    return fsolve(f1, x0=0.2, xtol=0.0001)[0]

In [18]:
# Compute the implied volatility for all portfolios
implied_volatilities = []
for _, port in portfolio.iterrows():
    if port['Type'] == 'Stock': # Stock
        implied_volatilities.append(None)
    else: # Option
        # Compute the Implied Volatility
        T = ((port['ExpirationDate']-curr_date).days)/365
        K, price, option_type = port['Strike'], port['CurrentPrice'], port['OptionType']
        dividend_time = int((dividend_date-curr_date).days/(port['ExpirationDate']-curr_date).days*N)
        sigma = compute_implied_volatility_binomial_tree(S0, K, T, r, N, option_type, price, [dividend_time], [dividend])
        implied_volatilities.append(sigma)

# Store the implied volatility in portfolios
portfolio['Implied Volatility'] = implied_volatilities
portfolio

/Users/wuyilun/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,Implied Volatility
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.455948
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.455948
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299
5,CallSpread,Option,AAPL,-1,Call,2023-04-21,160.0,2.21,-0.016455
6,PutSpread,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.455948
7,PutSpread,Option,AAPL,-1,Put,2023-04-21,140.0,1.84,0.421340
8,Stock,Stock,AAPL,1,NaN,NaT,NaN,151.03,NaN
9,Call,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299


In [19]:
# Compute the portfolio value over a range of underlying values
def compute_portfolio_value(portfolios, S0, num_days=0):
    portfolio_values = pd.DataFrame(index=portfolio_names, columns=[S0])
    portfolio_values = portfolio_values.fillna(0)
    for _, portfolio in portfolios.iterrows():
        if portfolio['Type'] == 'Stock': # Stock
            current_price = S0
        else: # Option
            T = ((portfolio['ExpirationDate']-curr_date).days-num_days)/365
            K, option_type = portfolio['Strike'], portfolio['OptionType']
            sigma = portfolio['Implied Volatility']
            current_prices = []
            for S in np.atleast_1d(S0):
                dividend_time = int((dividend_date-curr_date).days/(port['ExpirationDate']-curr_date).days*N)
                option_values = (binominal_tree_with_dividends(S, K, r, T, sigma, N, option_type, [dividend_time], [dividend]))
                current_prices.append(option_values)
            current_price = np.array(current_prices)
        # Compute the portfolio value with payoff
        portfolio_values.loc[portfolio['Portfolio'], :] += portfolio['Holding']*current_price
    
    return portfolio_values

In [20]:
# Fit a Normal distribution model to the log returns of AAPL
mean, std = norm.fit(AAPL_returns)
print(f"Mean: {mean}, Std: {std} of Normal distribution model to the log returns of AAPL")

Mean: -9.23320559794107e-19, Std: 0.022380778943343265 of Normal distribution model to the log returns of AAPL


In [21]:
# Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price
num_steps = 10
num_simulation = 1000
returns = AAPL_returns.values
returns_sim = np.random.normal(mean, std, (num_steps, num_simulation))
prices_sim = S0*np.exp(returns_sim.cumsum(axis=0))
prices_sim = pd.DataFrame(prices_sim)
prices_sim

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,160.253407,163.646133,162.221579,169.840535,166.168649,167.509141,168.399725,166.824409,163.093001,176.265052,...,171.638110,165.504003,163.822231,162.882222,164.173824,168.051276,163.227793,166.206690,165.488206,165.761242
1,152.536287,164.320342,167.980623,172.012842,163.419472,170.220276,171.020284,162.957562,166.593096,179.835776,...,174.456744,168.835367,159.951371,157.952229,167.514068,164.195943,165.851162,163.540686,166.672113,164.077947
2,149.220339,167.472517,167.346471,168.479528,159.958690,170.063663,168.654721,162.099229,168.943520,181.324679,...,181.291003,167.797036,161.227937,156.326394,170.847371,167.356081,166.700206,162.362455,166.407691,164.447547
3,149.588813,164.295959,169.822409,165.209258,158.515604,175.677036,173.569252,160.155988,169.343951,185.327771,...,179.727638,168.855475,162.757962,153.097145,171.682580,167.134455,163.440169,162.135715,165.427385,166.579604
4,150.252293,164.492354,170.574336,167.201726,160.428947,179.760407,171.870744,159.107849,166.485792,188.141397,...,182.620502,164.830989,161.850230,153.852191,174.276347,173.107393,168.403622,163.117381,163.432331,159.821469
5,157.412637,166.796749,168.301251,162.517107,163.731433,178.270008,172.628561,154.529936,171.442516,179.316885,...,185.127208,160.480006,163.553401,151.936705,172.174308,169.014146,170.129423,163.777023,165.359163,158.101762
6,156.237880,164.467214,170.573564,161.856159,162.120664,177.425610,170.350043,157.633756,166.695234,179.578822,...,184.245892,158.909211,161.639779,156.518034,177.474522,172.383004,164.810016,161.958163,170.852320,166.430244
7,156.833163,159.654592,169.730569,164.641205,162.841416,175.772806,167.718589,164.199239,169.139387,178.313788,...,186.976939,155.845175,160.476277,154.802316,179.212941,170.506683,171.894046,165.103362,169.026967,170.373017
8,160.852799,156.134509,171.316653,167.989734,159.823737,176.023093,165.282427,165.983473,167.493835,173.266819,...,188.360857,151.737616,168.309708,147.850256,177.915077,165.493786,177.842450,167.578489,165.487989,172.506569
9,158.937106,155.318405,170.631659,169.885728,161.960783,176.719416,168.100175,164.833157,173.911733,172.668080,...,192.975580,146.460493,168.342278,149.640360,182.217981,164.332050,180.226051,167.300785,170.588807,170.575690


In [22]:
# Compute the current portfolio value and the simulated portfolio value 10 days ahead
prices_sim_curr = prices_sim.loc[num_steps-1:].values[0]
portfolio_values_sim = compute_portfolio_value(portfolio, prices_sim_curr, num_days=num_steps)
portfolio_values_curr = compute_portfolio_value(portfolio, S0, num_days=0)

# Then compute Mean, VaR and ES 
portfolio_values_both = pd.merge(portfolio_values_sim, portfolio_values_curr, left_index=True, right_index=True)
portfolio_values_changes = portfolio_values_both.sub(portfolio_values_both[S0], axis=0).drop(S0, axis=1)
portfolio_metrics = pd.DataFrame(index=portfolio_names, columns=['Mean', 'VaR', 'ES'])
portfolio_metrics = portfolio_metrics.fillna(0)
for portfolio_name in portfolio_names:
    portfolio_values = portfolio_values_changes.loc[portfolio_name, :]
    mean = portfolio_values.mean()
    VaR = compute_VaR(portfolio_values)
    ES = compute_ES(portfolio_values)
    portfolio_metrics.loc[portfolio_name, 'Mean'] = mean
    portfolio_metrics.loc[portfolio_name, 'VaR'] = VaR
    portfolio_metrics.loc[portfolio_name, 'ES'] = ES

portfolio_metrics

,Mean,VaR,ES
Straddle,1.907300,9.093464,9.300737
SynLong,-0.199850,21.922218,25.836500
CallSpread,-1.881371,9.851777,9.965620
PutSpread,0.590558,1.077219,1.111019
Stock,0.329614,18.797547,24.109966
Call,0.853725,15.034248,15.148092
Put,1.053575,1.227297,1.261801
CoveredCall,-1.227020,8.621601,13.933258
ProtectedPut,1.086864,14.359546,16.426178


#### Calculate Mean, VaR and ES using Delta Normal

In [23]:
# First compute delta for each portfolio
delta_func = compute_partial_derivative(binominal_tree_with_dividends, 1, 'S')

deltas = []
for _, port in portfolio.iterrows():
    if port['Type'] == 'Stock': # Stock
        deltas.append(dividend*port['Holding'])
    else: # Option
        # Compute the Implied Volatility
        T = ((port['ExpirationDate']-curr_date).days)/365
        K, price, option_type = port['Strike'], port['CurrentPrice'], port['OptionType']
        dividend_time = int((dividend_date-curr_date).days/(port['ExpirationDate']-curr_date).days*N)
        sigma = port['Implied Volatility']
        delta = delta_func(S, K, T, r, sigma, N, option_type, [dividend_time], [dividend])
        deltas.append(delta)

# Store the delta information in portfolios
portfolio['Delta'] = deltas
portfolio

,Portfolio,Type,Underlying,Holding,OptionType,ExpirationDate,Strike,CurrentPrice,Implied Volatility,Delta
0,Straddle,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299,0.605779
1,Straddle,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.455948,-0.433511
2,SynLong,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299,0.605779
3,SynLong,Option,AAPL,-1,Put,2023-04-21,150.0,4.85,0.455948,-0.433511
4,CallSpread,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299,0.605779
5,CallSpread,Option,AAPL,-1,Call,2023-04-21,160.0,2.21,-0.016455,0.000000
6,PutSpread,Option,AAPL,1,Put,2023-04-21,150.0,4.85,0.455948,-0.433511
7,PutSpread,Option,AAPL,-1,Put,2023-04-21,140.0,1.84,0.421340,-0.284975
8,Stock,Stock,AAPL,1,NaN,NaT,NaN,151.03,NaN,1.000000
9,Call,Option,AAPL,1,Call,2023-04-21,150.0,6.80,-0.114299,0.605779


In [24]:
# Then compute Mean, VaR and ES 
alpha = 0.05
t = 10
portfolio_metrics = pd.DataFrame(index=portfolio_names, columns=['Mean', 'VaR', 'ES'])
for portfolio_name, port in portfolio.groupby('Portfolio'):
    gradient = S / port['CurrentPrice'].sum() * (port['Holding'] * port['Delta']).sum()
    port_std = abs(gradient)*std*np.sqrt(t)
    norm_dist = norm(0, 1)
    pv = port['CurrentPrice'].sum()
    portfolio_metrics.loc[portfolio_name]['Mean'] = 0 # mean of delta hedged portfolio is 0
    portfolio_metrics.loc[portfolio_name]['VaR'] = -pv*norm_dist.ppf(alpha)*port_std
    portfolio_metrics.loc[portfolio_name]['ES'] = pv*port_std*norm_dist.pdf(norm_dist.ppf(alpha))/alpha

portfolio_metrics

,Mean,VaR,ES
Straddle,0,3.028787,3.798221
SynLong,0,18.272682,22.91468
CallSpread,0,10.650734,13.356451
PutSpread,0,2.611549,3.274987
Stock,0,17.581895,22.048405
Call,0,10.650734,13.356451
Put,0,7.621947,9.55823
CoveredCall,0,12.893474,16.168937
ProtectedPut,0,10.955516,13.73866


### Problem 3

In [25]:
# Read data from csv file
research_data = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', parse_dates=['Date']).set_index('Date')
research_data

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2023-01-25,0.00,-0.04,0.65,0.017
2023-01-26,1.08,-0.58,0.01,0.017
2023-01-27,0.36,0.62,-1.16,0.017


In [26]:
momentum = pd.read_csv('F-F_Momentum_Factor_daily.CSV', parse_dates=['Date']).set_index('Date').rename(columns={'Mom   ':  "Mom"})
momentum

,Mom
Date,
1926-11-03,0.56
1926-11-04,-0.50
1926-11-05,1.17
1926-11-06,-0.03
1926-11-08,-0.01
...,...
2023-01-25,0.14
2023-01-26,-1.23
2023-01-27,-2.46


In [27]:
# Convert units and select data from past 10 years
factor = (research_data.join(momentum, how='right')/100).loc['2013-1-31':]
factor

,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,
2013-01-31,-0.0008,0.0069,0.0017,0.00000,-0.0037
2013-02-01,0.0099,0.0007,0.0024,0.00000,0.0031
2013-02-04,-0.0119,-0.0014,-0.0017,0.00000,0.0000
2013-02-05,0.0107,-0.0008,0.0014,0.00000,0.0013
2013-02-06,0.0015,0.0028,0.0014,0.00000,-0.0032
...,...,...,...,...,...
2023-01-25,0.0000,-0.0004,0.0065,0.00017,0.0014
2023-01-26,0.0108,-0.0058,0.0001,0.00017,-0.0123
2023-01-27,0.0036,0.0062,-0.0116,0.00017,-0.0246


In [28]:
DailyPrices = pd.read_csv('DailyPrices.csv', parse_dates=['Date'])
DailyReturns = pd.DataFrame(return_calculate(DailyPrices)).set_index('Date')
DailyReturns

/Users/wuyilun/Desktop/FINTECH545/lib/riskmgmt.py:236: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[vars[i]] = p2[:, i]


,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,BRK-B,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-15,0.016127,0.023152,0.018542,0.008658,0.053291,0.007987,0.008319,0.015158,0.091812,0.006109,...,0.012807,-0.004082,0.004592,0.052344,0.003600,-0.012275,0.033021,0.026240,0.028572,0.013237
2022-02-16,0.001121,-0.001389,-0.001167,0.010159,0.001041,0.008268,0.007784,-0.020181,0.000604,-0.001739,...,0.006757,-0.002429,0.005763,0.038879,0.009294,0.012244,0.003363,0.015301,-0.001389,-0.025984
2022-02-17,-0.021361,-0.021269,-0.029282,-0.021809,-0.050943,-0.037746,-0.037669,-0.040778,-0.075591,-0.006653,...,-0.034949,0.005326,0.015017,-0.046988,-0.009855,0.004833,-0.030857,-0.031925,-0.033380,-0.028763
2022-02-18,-0.006475,-0.009356,-0.009631,-0.013262,-0.022103,-0.016116,-0.013914,-0.007462,-0.035296,0.003987,...,-0.000646,-0.000908,0.007203,-0.000436,-0.003916,-0.005942,-0.013674,-0.004506,-0.003677,0.015038
2022-02-22,-0.010732,-0.017812,-0.000729,-0.015753,-0.041366,-0.004521,-0.008163,-0.019790,-0.010659,-0.002033,...,0.009494,0.007121,-0.008891,0.003243,-0.001147,-0.000673,0.008342,-0.037654,-0.002246,-0.013605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-03,-0.010629,0.024400,-0.023621,-0.084315,0.009083,-0.027474,-0.032904,-0.011866,-0.028053,-0.010742,...,-0.004694,-0.011251,-0.001277,-0.002677,0.038211,0.004134,0.002336,-0.008916,-0.005954,0.001617
2023-02-06,-0.006111,-0.017929,-0.006116,-0.011703,0.025161,-0.017942,-0.016632,-0.002520,-0.000521,-0.000259,...,-0.014451,0.003945,0.001066,-0.007102,0.022012,0.021826,-0.041181,0.005106,-0.009782,-0.004595
2023-02-07,0.013079,0.019245,0.042022,-0.000685,0.010526,0.046064,0.044167,0.029883,0.051401,0.014720,...,-0.000368,-0.016473,-0.008518,0.019544,-0.003590,-0.001641,0.003573,0.001451,0.008669,-0.003618


In [29]:
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
stockReturn = DailyReturns[stocks].join(factor)
stockReturn.dropna(inplace=True)
stockReturn

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,...,DIS,GOOGL,JNJ,BAC,CSCO,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-15,0.023152,0.015158,0.008073,0.019724,0.018542,0.091812,0.004836,-0.000201,0.008658,0.006109,...,0.025655,0.007987,0.010326,0.007803,0.020496,0.0187,0.0133,-0.0142,0.00000,-0.0091
2022-02-16,-0.001389,-0.020181,0.003806,0.003643,-0.001167,0.000604,-0.008974,-0.002209,0.010159,-0.001739,...,0.010535,0.008268,-0.000598,-0.002302,-0.000369,-0.0002,-0.0009,0.0031,0.00000,0.0064
2022-02-17,-0.021269,-0.040778,-0.020227,-0.024103,-0.029282,-0.075591,-0.006141,-0.015700,-0.021809,-0.006653,...,-0.021746,-0.037746,-0.006100,-0.033767,0.028018,-0.0228,-0.0028,0.0110,0.00000,0.0103
2022-02-18,-0.009356,-0.007462,-0.005379,-0.010035,-0.009631,-0.035296,-0.003075,-0.007566,-0.013262,0.003987,...,-0.010396,-0.016116,-0.010719,-0.002388,0.025820,-0.0087,-0.0009,0.0093,0.00000,0.0104
2022-02-22,-0.017812,-0.019790,-0.011329,-0.004487,-0.000729,-0.010659,-0.088506,-0.020606,-0.015753,-0.002033,...,-0.021604,-0.004521,-0.013590,-0.008703,-0.015906,-0.0118,-0.0048,0.0011,0.00000,0.0043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-25,-0.004701,-0.011457,0.001831,0.006263,-0.005908,0.003011,-0.001353,0.008052,0.008929,0.001927,...,0.020000,-0.025384,0.007130,0.008678,0.003556,0.0000,-0.0004,0.0065,0.00017,0.0014
2023-01-26,0.014803,0.040989,-0.000041,-0.013468,0.030714,0.024789,-0.010874,-0.009180,0.020992,-0.003046,...,0.014613,0.024155,-0.003658,0.013479,0.007503,0.0108,-0.0058,0.0001,0.00017,-0.0123
2023-01-27,0.013684,0.030143,-0.013056,-0.008509,0.000645,0.028431,0.009178,-0.010395,0.030437,-0.005724,...,-0.001458,0.018971,-0.003908,0.003113,0.003517,0.0036,0.0062,-0.0116,0.00017,-0.0246


In [30]:
# Compute alpha and beta
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
X = stockReturn[factors]
X = sm.add_constant(X)

y = stockReturn[stocks].sub(stockReturn['RF'], axis=0)
betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])

for stock in stocks:
    ols = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = ols.params[factors]
    alphas.loc[stock] = ols.params['const']

In [31]:
# Compute the expected annual return
returns = pd.DataFrame(np.dot(factor[factors], betas.T), index=factor.index, columns=betas.index)
merged_returns = pd.merge(returns, factor['RF'], left_index=True, right_index=True)
expected_daily_return = merged_returns.add(merged_returns['RF'], axis=0).drop('RF', axis=1).add(alphas.T.loc['Alpha'], axis=1)
expected_annual_return = ((expected_daily_return+1).cumprod().tail(1)**(1/expected_daily_return.shape[0])-1)*252
expected_annual_return

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
Date,,,,,,,,,,,,,,,,,,,,
2023-01-31,0.157144,0.017941,0.2538,0.222901,0.155944,0.279721,0.120591,0.076962,-0.042945,0.129923,0.08154,0.521821,-0.033253,0.098273,0.241054,-0.155372,-0.017075,0.124206,-0.112301,0.147807


In [32]:
# Construct an annual covariance matrix for the 10 stocks.
cov_matrix = stockReturn[stocks].cov()*252
cov_matrix

,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
AAPL,0.127313,0.132896,0.040927,0.083163,0.103414,0.173948,0.067017,0.033573,0.124040,0.056686,0.037937,0.039287,0.158104,0.059780,0.073316,0.088495,0.109853,0.023111,0.067044,0.066930
META,0.132896,0.350657,0.031386,0.103881,0.132129,0.234835,0.091591,0.044397,0.179064,0.061134,0.035664,0.028666,0.169091,0.076354,0.088387,0.120601,0.162237,0.020323,0.088145,0.073841
UNH,0.040927,0.031386,0.059467,0.032369,0.040153,0.050123,0.029262,0.033484,0.040635,0.029404,0.028593,0.025776,0.041942,0.034082,0.030200,0.025233,0.036141,0.024019,0.036335,0.030980
MA,0.083163,0.103881,0.032369,0.098243,0.081517,0.140236,0.058047,0.034542,0.098091,0.048617,0.031936,0.031646,0.100168,0.059946,0.084674,0.078914,0.079580,0.017769,0.064977,0.052845
MSFT,0.103414,0.132129,0.040153,0.081517,0.126169,0.174861,0.070630,0.035592,0.131217,0.053063,0.035356,0.033042,0.133154,0.058319,0.070107,0.087375,0.116166,0.020189,0.065942,0.061005
NVDA,0.173948,0.234835,0.050123,0.140236,0.174861,0.405163,0.113089,0.047774,0.221002,0.085758,0.043662,0.056995,0.294271,0.101550,0.121274,0.158885,0.186411,0.021465,0.115212,0.100403
HD,0.067017,0.091591,0.029262,0.058047,0.070630,0.113089,0.097115,0.033607,0.093834,0.043067,0.035180,0.018140,0.079395,0.045264,0.051850,0.063803,0.066678,0.022350,0.047361,0.048495
PFE,0.033573,0.044397,0.033484,0.034542,0.035592,0.047774,0.033607,0.072076,0.037099,0.032123,0.028224,0.021048,0.023306,0.032843,0.031652,0.024637,0.029597,0.028109,0.031559,0.030132
AMZN,0.124040,0.179064,0.040635,0.098091,0.131217,0.221002,0.093834,0.037099,0.235455,0.066123,0.031263,0.041641,0.191243,0.074303,0.086239,0.122761,0.143324,0.021989,0.087201,0.070422
BRK-B,0.056686,0.061134,0.029404,0.048617,0.053063,0.085758,0.043067,0.032123,0.066123,0.051027,0.025596,0.034613,0.063761,0.048276,0.043158,0.052239,0.055297,0.019728,0.052049,0.040977


In [33]:
# Assume the risk free rate is 0.0425. Find the super efficient portfolio.
def compute_super_efficient_portfolio(returns, r, cov_matrix):
    if len(returns.shape) == 1:
        num_assets = returns.shape[0]
    else:
        num_assets = returns.shape[1]
    
    # Objective function: maximize Sharpe ratio
    def neg_sharpe_ratio(weights):
        sharpe_ratio = (np.sum(returns*weights)-r) / np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -sharpe_ratio
    
    # Constraints
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    
    # Weights are between 0 and 1
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Solve 
    init_weights = np.ones(num_assets) / num_assets  # start with equal weights
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    # Return optimal weights and Sharpe ratio of resulting portfolio
    opt_weights = opt_result.x
    opt_port_return = np.sum(returns * opt_weights)
    opt_port_std_dev = np.sqrt(np.dot(opt_weights.T, np.dot(cov_matrix, opt_weights)))
    opt_sharpe_ratio = (opt_port_return - r) / opt_port_std_dev
    return opt_weights*100, opt_sharpe_ratio

r = 0.0425
weights, sharpe_ratio = compute_super_efficient_portfolio(expected_annual_return.values[0], r, cov_matrix)
print(f"The Portfolio's Sharpe Ratio is {sharpe_ratio}")
weights = pd.DataFrame(weights, index=expected_annual_return.columns, columns=['weight %']).round(2).T
weights

The Portfolio's Sharpe Ratio is 1.4682873413630941


,AAPL,META,UNH,MA,MSFT,NVDA,HD,PFE,AMZN,BRK-B,PG,XOM,TSLA,JPM,V,DIS,GOOGL,JNJ,BAC,CSCO
weight %,0.0,0.0,26.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.78,0.0,0.0,12.19,0.0,0.0,3.08,0.0,0.0
